In [14]:
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


from sklearn.metrics import confusion_matrix
from GetScoreSepsis import GetScoreSepsis

# Načtení dat

In [15]:
# Cesta k tabulce s pacienty
filepath = './dataSepsis.csv'

# Načtení celé tabulky
fullSepsis = pd.read_csv(filepath, sep=';')

# Odstranění sloupců a vymazání řádků s příliš mnoha NaN

In [16]:
# Seznam všech sloupců, které mají být odstraněny
# Sloupce jsem vybírali na základě počtu NaN, korelace a statistické významnosti

drop_list = ['Temp','EtCO2','BaseExcess','HCO3','FiO2','pH',
 'PaCO2','SaO2','AST','Alkalinephos','Chloride','Bilirubin_direct',
 'Lactate','Phosphate','Bilirubin_total','TroponinI','PTT','Fibrinogen','Unit1',
 'Unit2', 'SBP', 'DBP', 'Hct', 'Age', 'Platelets', 'BUN']

# Odstranění sloupců
relevantSepsis = fullSepsis.drop(columns=drop_list)

# Odstranění řádků, které mají polovinu a více hodnot jako NaN hodnoty
relevantSepsis = relevantSepsis.dropna(thresh=relevantSepsis.shape[1]/2)
relevantSepsis = relevantSepsis.reset_index()
relevantSepsis.drop(columns=['index'], inplace=True)

# Separování sloupce s určením, zda je pacient sepický či nikoliv

In [17]:
isSepsis = relevantSepsis["isSepsis"]
relevantSepsis.drop(columns=["isSepsis"], inplace=True)

# Předzpracování dat

In [18]:
df = relevantSepsis
# Výpočet interquartilového rozptylu pro každý sloupce
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Stanovení horní a dolní hranice
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Nahrazení odlehlých hodnot hodnotami NaN na základě daných hranic
df_outliers_removed = df[(df >= lower_bound) & (df <= upper_bound)]

# Standardizace datasetu (dojde k převedení na array)
scaler = StandardScaler()
standardizedSepsis = scaler.fit_transform(df_outliers_removed)

# Nahrazení NaN pomocí k-nejbližších sousedů
imputer = KNNImputer(n_neighbors=100)
finalSepsis = imputer.fit_transform(standardizedSepsis)

# Zpětné převedení na dataframe
preprocessed_data = pd.DataFrame(data= finalSepsis, columns=df.columns)

# Vytvoření dvou datasetů s náhodně vybranými pacienty se sepsí a bez sepse

In [19]:
sepsis_df = preprocessed_data[isSepsis==1].sample(2000)
no_sepsis_df = preprocessed_data[isSepsis==0].sample(2000)

# Trénování modelu

In [20]:
# Zpětné přidání informace o diagnostice sepse
sepsis_df['sepsis'] = 1
no_sepsis_df['sepsis'] = 0

# Spojení datasetů do testovacího datasetu
combined_df = pd.concat([sepsis_df, no_sepsis_df], ignore_index=True)

# Oddělení příznaků (X) od předpovídané proměnné (y)
X = combined_df.drop('sepsis', axis=1)
y = combined_df['sepsis']

# Rozdělení dat na testovací a trénovací množinu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Vytvoření XGBoost classifier
model = XGBClassifier(n_estimators=100,
                      max_depth=4,
                      max_leaves=4,
                      grow_policy="depthwise",
                      booster="gbtree",
                      early_stopping_rounds=10
                      )

model.fit(X_train, y_train, eval_set=[(X_test, y_test)])


[0]	validation_0-logloss:0.63645
[1]	validation_0-logloss:0.58238
[2]	validation_0-logloss:0.55620
[3]	validation_0-logloss:0.53729
[4]	validation_0-logloss:0.52312
[5]	validation_0-logloss:0.49770
[6]	validation_0-logloss:0.48701
[7]	validation_0-logloss:0.47430
[8]	validation_0-logloss:0.46465
[9]	validation_0-logloss:0.45852
[10]	validation_0-logloss:0.45379
[11]	validation_0-logloss:0.44250
[12]	validation_0-logloss:0.43746
[13]	validation_0-logloss:0.43231
[14]	validation_0-logloss:0.42304
[15]	validation_0-logloss:0.42065
[16]	validation_0-logloss:0.41662


[17]	validation_0-logloss:0.41484
[18]	validation_0-logloss:0.41357
[19]	validation_0-logloss:0.40811
[20]	validation_0-logloss:0.40654
[21]	validation_0-logloss:0.40488
[22]	validation_0-logloss:0.40008
[23]	validation_0-logloss:0.39533
[24]	validation_0-logloss:0.39475
[25]	validation_0-logloss:0.39362
[26]	validation_0-logloss:0.39405
[27]	validation_0-logloss:0.39000
[28]	validation_0-logloss:0.38902
[29]	validation_0-logloss:0.38752
[30]	validation_0-logloss:0.38484
[31]	validation_0-logloss:0.38328
[32]	validation_0-logloss:0.38236
[33]	validation_0-logloss:0.38270
[34]	validation_0-logloss:0.38117
[35]	validation_0-logloss:0.38098
[36]	validation_0-logloss:0.38006
[37]	validation_0-logloss:0.37990
[38]	validation_0-logloss:0.37824
[39]	validation_0-logloss:0.37893
[40]	validation_0-logloss:0.37920
[41]	validation_0-logloss:0.37901
[42]	validation_0-logloss:0.37772
[43]	validation_0-logloss:0.37631
[44]	validation_0-logloss:0.37599
[45]	validation_0-logloss:0.37590
[46]	validatio

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy='depthwise', importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=4,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

# Zjišťování score pro trénovací množinu

In [23]:
yPredTrain = model.predict(X_train)

confMatrixTrain = confusion_matrix(y_train, yPredTrain)
print(confMatrixTrain)
GetScoreSepsis(confMatrixTrain)

[[1404  218]
 [ 221 1357]]


(0.8615873015873016, 0.864, 0.8628125, 0.8599493029150824, 0.860767522993974)

# Zjišťování score pro testovací množinu

In [22]:
yPred = model.predict(X_test)

confMatrix = confusion_matrix(y_test, yPred)
GetScoreSepsis(confMatrix)

[[307  71]
 [ 62 360]]


(0.8352668213457076,
 0.8319783197831978,
 0.83375,
 0.8530805687203792,
 0.8440797186400938)

# Uložení modelu

In [47]:
joblib.dump(model, "HVH_model.joblib")

['HVH_model.joblib']